In [1]:
import pynvml
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
info = pynvml.nvmlDeviceGetMemoryInfo(handle)
print(f'Total memory: {info.total / 1024**3:.2f} GB')
print(f'Free memory: {info.free / 1024**3:.2f} GB')
print(f'Used memory: {info.used / 1024**3:.2f} GB')
import dask_cudf
import cupy as cp
import rmm
import cudf
import gc
import cudf.pandas
cudf.pandas.install()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Total memory: 8.00 GB
Free memory: 7.48 GB
Used memory: 0.52 GB


In [2]:
mta_station_data = dask_cudf.read_csv('MTA_Subway_Stations.csv', blocksize="100MB")


In [3]:
mta_station_data.columns

Index(['GTFS Stop ID', 'Station ID', 'Complex ID', 'Division', 'Line',
       'Stop Name', 'Borough', 'Daytime Routes', 'Structure', 'GTFS Latitude',
       'GTFS Longitude', 'North Direction Label', 'South Direction Label',
       'ADA', 'ADA Northbound', 'ADA Southbound', 'ADA Notes', 'Georeference'],
      dtype='object')

In [4]:
mta_station_data = mta_station_data[['Station ID', 'Line', 'Stop Name', 'Borough', 'GTFS Latitude', 'GTFS Longitude']]

In [5]:
rows = mta_station_data.shape[0].compute()
print(rows)

496


In [6]:
for column in mta_station_data.columns:
    print(f"Value counts for {column}:")
    value_counts = mta_station_data[column].value_counts().compute()
    print(value_counts)
    print("\n")
    # print(f"Unique values for {column}:")
    
    # Replace null values with a placeholder or drop them
    # taxi_busyness_data[column] = taxi_busyness_data[column].fillna(-1)  # Adjust based on data type and context
    # print("hi")
    
    # unique_values = taxi_busyness_data[column].unique_count()
    # print(unique_values)
    # print("\n")

print(mta_station_data.nunique().compute())
info = pynvml.nvmlDeviceGetMemoryInfo(handle)

print(f'Used memory: {info.used / 1024**3:.2f} GB')


Value counts for Station ID:
Station ID
151    2
167    2
461    2
1      1
2      1
      ..
517    1
518    1
519    1
522    1
523    1
Name: count, Length: 493, dtype: int64


Value counts for Line:
Line
8th Av - Fulton St                          46
Broadway - 7Av                              38
6th Av - Culver                             31
Broadway - Brighton                         28
Jamaica                                     28
Lenox - White Plains Rd                     26
Canarsie                                    24
Queens Blvd                                 24
Lexington Av                                23
Flushing                                    22
Staten Island                               21
Eastern Pky                                 18
Pelham                                      18
Jerome Av                                   14
Rockaway                                    14
4th Av                                      13
Concourse                               

In [7]:
unique_mta_station_data = mta_station_data.drop_duplicates(subset=['Station ID'], keep='first')

In [9]:
print(f"Value counts for Station ID:")
value_counts = unique_mta_station_data["Station ID"].value_counts().compute()
print(value_counts)

Value counts for Station ID:
Station ID
1      1
2      1
3      1
4      1
5      1
      ..
517    1
518    1
519    1
522    1
523    1
Name: count, Length: 493, dtype: int64


In [10]:
unique_mta_station_data.to_csv('saved_csvs/mta_stations.csv', single_file=True)

['/home/eamonn-walsh/Documents/Summer-Project/saved_csvs/mta_stations.csv']